In [3]:
import pandas as pd
import statsmodels.api as sm
import matplotlib.pyplot as plt
import numpy as np
from glob import glob

# Funções

In [68]:
def save_summary(result, year, pattern):
    pattern = pattern.split('/')[-1].split('.')[0].split('*')[0]
    with open(f'Summary/ATE-{pattern}{year}.txt', 'w') as f:
        print(f'REGRESSÃO PARA {year}', file=f)
        print(result.summary(), file=f)

In [65]:
def run_linear(df, y_col, X, year):
  lm = sm.WLS(df[y_col], sm.add_constant(df[X]), weights=df['weights'], missing='drop')
  res = lm.fit(cov_type='cluster', cov_kwds={'groups': df['subclass']})
  return res

In [66]:
def read_files(pattern):
    files = glob(pattern)
    df = pd.concat([pd.read_parquet(f) for f in files])
    return df

In [64]:
def calculate_ate(pattern, y_col, X):
    df = read_files(pattern)
    for year in df.NU_ANO.unique():
        sel = df[df.NU_ANO == year]
        sel = sel[(~sel.isin([np.inf, -np.inf]).any(1))]
        res = run_linear(sel, y_col, X, year)
        save_summary(res, year, pattern)

# Script

Avaliando Avarage Treatment Effect Para Internet

In [69]:
file = "../../Resultados/Agg/ENEM/aux/postMatching/individuo-Q025-*.parquet"
calculate_ate(file, 'LOG_NT_MEDIA', 'Q025')

Avaliando Avarage Treatment Effect para alunos de escola pública e Internet

In [70]:
file = "../../Resultados/Agg/ENEM/aux/postMatching/individuo-Q025-publica*.parquet"
calculate_ate(file, 'LOG_NT_MEDIA', 'Q025')

Avaliando ATE para COMPUTADOR

In [ ]:
file = "../../Resultados/Agg/ENEM/aux/postMatching/individuo-Q024-*.parquet"
calculate_ate(file, 'LOG_NT_MEDIA', 'Q024')